In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import Model
import numpy as np
from matplotlib import pyplot as plt
import tensorboard
%load_ext tensorboard

In [2]:
def get_data():
    #filename = os.path.abspath(os.path.join(os.path.dirname( __file__ ), 'data', 'processed', 'cases_with_population_and_hospital.csv'))
    filename = 'data/processed/timestamped_cases_with_population_and_hospital.csv'
    data = np.genfromtxt(filename, delimiter=',')
    np.random.shuffle(data)
    return data

def get_sliced_data(data):
    result = {}
    train_x1 = data[:10000, :4]
    result['train_x1'] = train_x1[:, :, np.newaxis]
    result['train_x2'] = data[:10000, 4:13]
    result['train_y'] = data[:10000, 13]
    test_x1 = data[260000:, :4]
    result['test_x1'] = test_x1[:, :, np.newaxis]
    result['test_x2'] = data[260000:, 4:13]
    result['test_y'] = data[260000:, 13]

    return result

In [3]:
datas = get_sliced_data(get_data())

In [4]:
def make_model():
    inp1 = Input(shape=(4, 1))
    inp2 = Input(shape=(9,))

    x = LSTM(64, activation='relu')(inp1)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(8, activation='relu')(x)

    y = Dense(64, activation='relu')(inp2)
    y = Dropout(0.5)(y)
    y = Dense(32, activation='relu')(y)
    y = Dropout(0.5)(y)
    y = Dense(16, activation='relu')(y)
    y = Dropout(0.5)(y)
    y = Dense(8, activation='relu')(y)

    z = Concatenate()([x, y])
    z = Dense(16, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(8, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(4, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(2, activation='relu')(z)

    out = Dense(1)(z)

    model = Model(inputs=[inp1, inp2], outputs=out)
    
    model.compile(optimizer='adam',
                  loss=MeanSquaredError(),
                  metrics=['mean_squared_error'])
    
    return model

def make_call_back():
    checkpoint_path = 'model/model.ckpt'
    return tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)


def fit_model(model, datas, epochs, cp_callback=False, weights=False, checkpoint_path='model/model.ckpt'):
    inp1 = datas['train_x1']
    inp2 = datas['train_x2']
    out = datas['train_y']

    logdir="logs/fit/"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    if weights:
        model.load_weights(checkpoint_path)
    
    if cp_callback:
        history = model.fit([inp1, inp2], [out], epochs=epochs, callbacks=[make_call_back(), tensorboard_callback])
    else:
        history = model.fit([inp1, inp2], [out], epochs=epochs)
    
    return model, history


model = make_model()

In [5]:
def make_plot(history):
    plt.plot(history.history['mean_squared_error'])
    plt.show()


def test_model(model, x1, x2, y):
    test_loss, test_acc = model.evaluate([x1, x2], y, verbose=2)
    return test_loss, test_acc

In [ ]:
model, history = fit_model(model, datas, 100, cp_callback=True, weights=True)

In [6]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1348), started 0:06:01 ago. (Use '!kill 1348' to kill it.)

In [ ]:
make_plot(history)

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png')